In [1]:
import PyPDF2
import fitz  # PyMuPDF
from PIL import Image
import pytesseract
import os
import re

In [2]:
pdf_file_path = 'Voter List.pdf' # change file name to what you want to replace with
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'


In [3]:
pdf_document = fitz.open(pdf_file_path)
all_extracted_text = []

for page_num in range(pdf_document.page_count):
    page = pdf_document[page_num]  # Get the page
    
    # Extract images from the page
    images = page.get_images(full=True)

    
    for img_index, img in enumerate(images):
        xref = img[0]
        base_image = pdf_document.extract_image(xref)
        image_data = base_image['image']
        
        # Save or process the image data as needed
        # For example, you can save the image as a file:
        with open(f'page_{page_num + 1}_img_{img_index}.png', 'wb') as img_file:
            img_file.write(image_data)
    for img_index, img in enumerate(images):
        image_path = f'page_{page_num + 1}_img_{img_index}.png'
        image = Image.open(image_path)
        extracted_text = pytesseract.image_to_string(image,  lang ="eng+hin")
        all_extracted_text.append(extracted_text)

# Combine all extracted text into a single string
combined_text = '\n'.join(all_extracted_text)
#         print(f'Text from {image_path}:')
#         print(extracted_text)


In [4]:
import re
modified_text = combined_text.replace("लिंग ;", "लिंग:").replace("am:", "नाम:").replace("नामः","नाम:").replace("Photo is","")
modified_text =modified_text.replace("जाम:", "नाम:").replace("मकानसंख्या", "मकान संख्या").replace("लिग","लिंग")
modified_text =modified_text.replace("fem:", "लिंग:").replace("ar:", "नाम:").replace("AM:", "नाम:")


# Define a regular expression pattern to match the repeating phrase
pattern = r'((नाम:|पिता का नाम:|पति का नाम:|मकान संख्या :|आयु :|नामः).*?)(?=(नाम:|पिता का नाम:|पति का नाम:|मकान संख्या :|आयु :|नामः)|$)'

# Use re.sub() to split lines when the phrase repeats
result_text = re.sub(pattern, r'\1\n', modified_text)
# Print the result
pattern = r'(आयु :.*?)(?=लिंग:|$)'

# Use re.sub() to split lines when the phrases repeat
result_text = re.sub(pattern, r'\1\n', result_text)

pattern = r'(आयु:.*?)(?=लिंग:|$)'

result_text = re.sub(pattern, r'\1\n', result_text)

pattern = r'(लिंग:.*?)(?=आयु :|$)'
result_text = re.sub(pattern, r'\1\n', result_text)
pattern = r'(मकान संख्या : (\d+/\d+))'
result_text = re.sub(pattern, r'\1\n', result_text)

pattern = r'(लिंग: (पुरुष|महिला)|$)'
result_text = re.sub(pattern, r'\1\n', result_text)
print(result_text)

विधान सभा क्षेत्र की संख्या व नाम : : 211-कल्याणपुर

अनुभाग संख्या व नाम : : 1-पी डब्लू डी हाउसिंग सोसायटी

Pt
नाम: अबू हसनात

पिता का नाम: अब्दुल ग़मख़्वार
मकान संख्या : 2/19
 टेलिफोन कॉलोनी
आयु : 23 
लिंग: पुरुष


NXB4046181

Phato is
Available

A |

2
नाम: शोमित प्रजापति
पिता का नाम: दाम बाबू
मकान संख्या : 14/372

आयु : 29 
लिंग: पुरुष


=
=
5
हर

Phato is
Available

भाग संख्या : 161

नाम: जय शंकर बाजपेयी

पिता का नाम: लक्ष्मी नारायण बाजपेयी
मकान संख्या : 20/02


आयु : 67 
लिंग: पुरुष


NXB3903119

Phato is
Available

; 4 |

नाम: सुमन देवी बाजपेयी

पति का नाम: जय शंकर बाजपेयी
मकान संख्या : 20/2


आयु : 66 
लिंग: महिला


NXB3903127

Phato is
Available

5
नाम: adit शुक्ला

पति का नाम: नागेंद्र प्रसाद शुक्ला
मकान संख्या : >जे

आयु : 64 
लिंग: महिला


Phato is
Available

|} [|
नाम: अमित शुक्ला

पिता का नाम: Pee प्रसाद शुक्ला
मकान संख्या : श जे

आयु : 43 
लिंग: पुरुष


Le
:

Phato is
Available

i
नाम: गायत्री शुक्ला

पति का नाम: अमित शुक्ला
मकान संख्या : 27 जे
आयु : 42 
लिंग: महिला


Pha

In [5]:
file_path = 'combined_text.txt'
# Open the file in write mode and save the combined text to it
with open(file_path, 'w',encoding='utf-8') as file:
    file.write(result_text)

In [6]:
import pandas as pd
regex = r'^नाम:(.*?)$'
name = re.findall(regex, result_text, re.MULTILINE)
#for name in names:
#     naam = name.strip()


regex = r'(पिता|पति) का नाम:(.*?)\n'
father_name = re.findall(regex, result_text)
# for match in father_name:
#     print(match)

regex = r'मकान संख्या :(.*?)\n'
house_number = re.findall(regex, result_text)


regex = r'आयु :(.*?)\n'
age = re.findall(regex, result_text)

regex = r'लिंग:(.*?)\n'
gender = re.findall(regex, result_text)
# for match in name:
#     print(match)

# Find the maximum length between father_name and name
max_length = max(len(father_name), len(name), len(house_number), len(age), len(gender))

# Fill the shorter lists with appropriate values to match the maximum length
if len(father_name) < max_length:
    father_name += [""] * (max_length - len(father_name))
if len(name) < max_length:
    name += [""] * (max_length - len(name))
if len(house_number) < max_length:
    house_number += [""] * (max_length - len(house_number))
if len(age) < max_length:
    age += [None] * (max_length - len(age))  # Fill with None for missing values
if len(gender) < max_length:
    gender += [None] * (max_length - len(gender))  # Fill with None for missing values

# Create the DataFrame
df = pd.DataFrame({ 'Name': name,'Father/Husband Name': father_name, 'House Number': house_number, 'Age': age, 'Gender': gender})
df.index = range(1, len(df) + 1)
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
# Save the DataFrame to an Excel file



In [7]:
df

,Name,Father/Husband Name,House Number,Age,Gender
1,अबू हसनात,"(पिता, अब्दुल ग़मख़्वार)",2/19,23,पुरुष
2,शोमित प्रजापति,"(पिता, दाम बाबू)",14/372,29,पुरुष
3,जय शंकर बाजपेयी,"(पिता, लक्ष्मी नारायण बाजपेयी)",20/02,67,पुरुष
4,सुमन देवी बाजपेयी,"(पति, जय शंकर बाजपेयी)",20/2,66,महिला
5,adit शुक्ला,"(पति, नागेंद्र प्रसाद शुक्ला)",>जे,64,महिला
6,अमित शुक्ला,"(पिता, Pee प्रसाद शुक्ला)",श जे,43,पुरुष
7,गायत्री शुक्ला,"(पति, अमित शुक्ला)",27 जे,42,महिला
8,बिनीत कुमार,"(पिता, प्रभाकर चंद्रा)",29,46,पुरुष
9,सुरेश चंद्र दिक्षित,"(पिता, जगजीवन प्रसाद दिक्षित)",35/116,66,पुरुष
10,रंजना विक्षित,"(पिता, सुरेश चंद्र विक्षित)",35/116,36,पुरुष


In [8]:
excel_file_path = 'data.xlsx'

# Save the DataFrame to the Excel file
df.to_excel(excel_file_path, index=False)

print(f'DataFrame saved to {excel_file_path}')

DataFrame saved to data.xlsx
